# Сборный проект 2

## Введение

HR-аналитики компании «Работа с заботой» помогают бизнесу оптимизировать управление персоналом: бизнес предоставляет данные, а аналитики предлагают, как избежать финансовых потерь и оттока сотрудников. В этом HR-аналитикам пригодится машинное обучение, с помощью которого получится быстрее и точнее отвечать на вопросы бизнеса.
Компания предоставила данные с характеристиками сотрудников компании. Среди них — уровень удовлетворённости сотрудника работой в компании. Эту информацию получили из форм обратной связи: сотрудники заполняют тест-опросник, и по его результатам рассчитывается доля их удовлетворённости от 0 до 1, где 0 — совершенно неудовлетворён, 1 — полностью удовлетворён. 
Существует две задачи:
- построить модель, которая сможет предсказать уровень удовлетворённости сотрудника на основе данных заказчика. 
- построить модель, которая сможет на основе данных заказчика предсказать то, что сотрудник уволится из компании.

## Задача 1: предсказание уровня удовлетворённости сотрудника

### Загрузка данных

### Предобработка данных

### Исследовательский анализ данных

### Подготовка данных

### Обучение моделей

### Выводы

## Задача 2: предсказание увольнения сотрудника из компании

### Загрузка данных

### Предобработка данных

### Исследовательский анализ данных

### Добавление нового входного признака

### Подготовка данных

### Обучение моделей

### Выводы

## Общие выводы